# Import the Libraries

In [ ]:
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import mpld3

from bokeh.io import show, save, output_file
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import DataTable, TableColumn

%matplotlib inline
mpld3.enable_notebook()

# Import the Data and construct the DataFrame

In [ ]:
data = pd.read_csv('Climate_and_Economy_Data.csv')
data['GDP_Per_Capita_PPP'] = round(data['GDP_Per_Capita_PPP']/1000,2)
data = data.rename(columns = {'GDP_Per_Capita_PPP': 'GDP Per Capita PPP (Thousand $ Year)', 'Rainfall': 'Rainfall (mm per month)',
                             'Temperature': 'Temperature (Cº)'})
data = data.set_index('ISO3')

## Scatter Plot - Labels when Hover

In [ ]:
fig, ax = plt.subplots(subplot_kw=dict(facecolor='#EEEEEE'), figsize=(20, 10))

#Define the Variables
x = 'Rainfall (mm per month)' #'Temperature (Cº)'
y = 'GDP Per Capita PPP (Thousand $ Year)'

x_metric = 'mm/pm' if x == 'Rainfall (mm per month)' else 'Cº'
y_metric = 'K USD/Year'

output_name = str(str(x.replace(" ", "_"))+'vs'+str(y.replace(" ", "_"))+'.html') 

# Function to map the colors as a list from the input list of x variables
def pltcolor(lst):
    cols=[]
    for l in lst:
        if l=='Europe':
            cols.append('lime')
        elif l=='Asia':
            cols.append('yellow')
        elif l=='Oceania':
            cols.append('red')
        elif l=='North America':
            cols.append('navy')
        elif l=='South America':
            cols.append('aqua')
        elif l=='Africa':
            cols.append('saddlebrown')
    return cols

# Create the colors list using the function above
data['Color'] = pltcolor(data['Continent'])

scatter = ax.scatter(data[x],
                     data[y],
                     c=data['Color'],
                     s=400,
                     alpha=0.5,
                     cmap=plt.cm.jet)

#Graph Feautures
ax.set_title(str(str(x)+' vs '+str(y)), size=20)
plt.xlabel(x)
plt.ylabel(y)

#Legend
green_patch = mpatches.Patch(color='lime', label='Europe')
yellow_patch = mpatches.Patch(color='yellow', label='Asia')
red_patch = mpatches.Patch(color='red', label='Oceania')
blue_patch = mpatches.Patch(color='navy', label='North America')
gray_patch = mpatches.Patch(color='aqua', label='South America')
brown_patch = mpatches.Patch(color='saddlebrown', label='Africa')
plt.legend(handles=[green_patch, yellow_patch, red_patch, blue_patch, gray_patch, brown_patch], fancybox=True, framealpha=0.5)

#Include the Trendline
z = np.polyfit(data[x], data[y], 1)
p = np.poly1d(z)
ax.plot(data[x],p(data[x]),"black")

#Prepare the Scatter dots label and get it bundled in tooltip
labels = []
for i in data.index:
    labels.append(str(data['Country'][i]) + '<br/>' + str(data[x][i]) + x_metric + '<br/>' + str(data[y][i]) + y_metric)
    
#bubble_label = np.asarray(data['Country'])
tooltip = mpld3.plugins.PointHTMLTooltip(scatter, labels)

#Show the Graph
mpld3.plugins.connect(fig, tooltip)

#mpld3.display()

#Save Graph
mpld3.save_html(fig,output_name)

# Save the table to html using Bokeh

In [ ]:
data_table_columns = ['Country', 'GDP Per Capita PPP (Thousand $ Year)', 'Temperature (Cº)', 'Rainfall (mm per month)']

source = ColumnDataSource(data)

columns = [TableColumn(field=Ci, title=Ci, width=20) for Ci in data_table_columns]

data_table = DataTable(source=source, columns=columns, width=800, height=280, selectable = True, index_position = None)

output_file('Climate_and_Economy_Growth_Table.html')
save(data_table)